# Uber H3 Demo
H3 is hierarchical spatial index solution that applies recursive decomposition based on hexagonal cells.

https://nbviewer.org/github/ozdemirht/py_redis_1/blob/master/docs/Test-H3.ipynb


## Setup
Need to install h3 and folium first

In [1]:
%pip install h3
%pip install folium
%pip install geopandas
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Helper Functions
This section will walk reader thorough the helper functions.

Let's first import packages this notebook will need.

In [2]:
import h3
import folium
from shapely.geometry import MultiPolygon, Polygon
from IPython.display import display
import pandas as pd

Let's display the H3 Cell id of a location at different hexagon resolutions.

In [3]:
# Let's explore H3Cell
df = pd.DataFrame(columns=["Lat", "Lon", "Hexagon Resolution", "H3 Address", "H3 Cell First Part", "H3 Cell Second Part"])
43.0375919,
test_lat = 37.3615593
test_lon = -122.0553238
test_lat = 43.0375919
test_lon = -76.135058
test_hexagon_resolutions = [7,8,9,10,11,12,13,14,15]
for i in test_hexagon_resolutions:
    hexagon_resolution=i
    h3_address = h3.latlng_to_cell(test_lat,test_lon, hexagon_resolution) # lat, lng, hex resolution
    print(f"H3 address ({test_lat},{test_lon},{hexagon_resolution:02d}): {h3_address} -> {h3_address[:3]} {h3_address[3:]}")
    new_row = {"Lat":test_lat, "Lon":test_lon, "Hexagon Resolution": hexagon_resolution, "H3 Address": h3_address, "H3 Cell First Part": h3_address[:3], "H3 Cell Second Part": h3_address[3:] }
    df.loc[len(df)]=new_row


H3 address (43.0375919,-76.135058,07): 872b8a2e3ffffff -> 872 b8a2e3ffffff
H3 address (43.0375919,-76.135058,08): 882b8a2e3dfffff -> 882 b8a2e3dfffff
H3 address (43.0375919,-76.135058,09): 892b8a2e3cfffff -> 892 b8a2e3cfffff
H3 address (43.0375919,-76.135058,10): 8a2b8a2e3cd7fff -> 8a2 b8a2e3cd7fff
H3 address (43.0375919,-76.135058,11): 8b2b8a2e3cd0fff -> 8b2 b8a2e3cd0fff
H3 address (43.0375919,-76.135058,12): 8c2b8a2e3cd0bff -> 8c2 b8a2e3cd0bff
H3 address (43.0375919,-76.135058,13): 8d2b8a2e3cd0bbf -> 8d2 b8a2e3cd0bbf
H3 address (43.0375919,-76.135058,14): 8e2b8a2e3cd0baf -> 8e2 b8a2e3cd0baf
H3 address (43.0375919,-76.135058,15): 8f2b8a2e3cd0bac -> 8f2 b8a2e3cd0bac


When the H3 Cell id of the same location at different resolution is displayed, it is easier to recognize the hierarchical spatial index encoding. For instance, the second character correlates with the resolution. In the last 12 characters, it is easier to observe that the prefix emerges as the resolution is increased.

In [4]:
from tabulate import tabulate
print(tabulate(df, headers = 'keys', tablefmt = 'psql'))

+----+---------+----------+----------------------+-----------------+----------------------+-----------------------+
|    |     Lat |      Lon |   Hexagon Resolution | H3 Address      | H3 Cell First Part   | H3 Cell Second Part   |
|----+---------+----------+----------------------+-----------------+----------------------+-----------------------|
|  0 | 43.0376 | -76.1351 |                    7 | 872b8a2e3ffffff | 872                  | b8a2e3ffffff          |
|  1 | 43.0376 | -76.1351 |                    8 | 882b8a2e3dfffff | 882                  | b8a2e3dfffff          |
|  2 | 43.0376 | -76.1351 |                    9 | 892b8a2e3cfffff | 892                  | b8a2e3cfffff          |
|  3 | 43.0376 | -76.1351 |                   10 | 8a2b8a2e3cd7fff | 8a2                  | b8a2e3cd7fff          |
|  4 | 43.0376 | -76.1351 |                   11 | 8b2b8a2e3cd0fff | 8b2                  | b8a2e3cd0fff          |
|  5 | 43.0376 | -76.1351 |                   12 | 8c2b8a2e3cd0bff | 8c2

In [5]:
# Extract the vertices from MultiPolygon structure, assuming 1 polygon
def get_outline_coordinates(mpoly: MultiPolygon):
    outline_coordinates = []
    for polygon in mpoly['coordinates']:
        if polygon:
            for vertices in polygon:
                if vertices:
                    for coord in vertices:
                        outline_coordinates.append([coord[0],coord[1]]) ## needs further investigation why it needs to be switched
    return outline_coordinates

In [6]:
# Defines line segments from list of vertices
# assumption: vertices are in order
def get_polyLines(coordinates):
    returnlines = []
    i=1;
    while i < len(coordinates):
        returnlines.append([coordinates[i-1],coordinates[i]])
        i=i+1
    return returnlines

In [7]:
# Switching
def reverseLatLonLinesOfPolygon(vertices):
    result = []
    [result.append([v[1], v[0]]) for v in vertices]
    return result

Method visualize_polygon() renders polygon a map (folium.Map). <br>
The returned map can be rendered by calling display(m)

In [8]:
###
def visualize_polygon(polyline, color):
    """
        Visualize a polygon by using folium
    :param polyline:
    :param color:
    :return:
    """
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='Cartodb Positron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m


Method visualize_hexagons() renders hexagons on a map (folium.Map)

In [44]:
###
def visualize_hexagons(hexagons, color="cyan", folium_map=None,zoom_start=13):
    """
    Visualize Hexagons by using folium tool

    :param hexagons: is a list of hexcluster. Each hexcluster is a list of hexagons. eg. [[hex1, hex2], [hex3, hex4]]
    :param color:
    :param folium_map:
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        #polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        polygons = h3.cells_to_geo([hex], tight=False)
        # flatten polygons into loops.
        #outlines = [loop for polygon in polygons for loop in polygon]
        outlines = get_outline_coordinates(polygons)
        #polyline = [outline + [outline[0]] for outline in outlines][0]
        polyline = get_polyLines(outlines)
        #polyline = h3.LatLngPoly(outlines)
        polyline = outlines
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)

    #for polyline in polylines:
    #    print(polyline)
    location=[sum(lng)/len(lng),sum(lat)/len(lat)]
    print(f"Center :{location}")

    if folium_map is None:
        #m = folium.Map(location=[sum(lng)/len(lng), sum(lat)/len(lat)], tiles='Cartodb Positron', zoom_start=13)
        m = folium.Map(location=[sum(lng)/len(lng), sum(lat)/len(lat)], tiles='OpenStreetMap', zoom_start=zoom_start)
    else:
        m = folium_map

    for polyline in polylines:
        #my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        my_PolyLine=folium.PolyLine(locations=reverseLatLonLinesOfPolygon(polyline),weight=8,color=color)
        m.add_child(my_PolyLine)
        for v in polyline:
            r=5
            circle = folium.CircleMarker(
                location=[v[1], v[0]],
                radius=r,
                color="cornflowerblue",
                stroke=False,
                fill=True,
                fill_opacity=0.6,
                opacity=1,
                popup="{} pixels".format(r),
                tooltip="{},{}".format(v[1], v[0]),
            )
            m.add_child(circle)

    return m


## H3 Cell Visualization on a Map
- It will translate location at resolution 9 to H3 Cell address
- Then finds the vertices of H3 Cell
- Displays the boundary of H3 Cell on a Map

In [10]:
hexagon_resolution=9
h3_address = h3.latlng_to_cell(test_lat,test_lon, hexagon_resolution)
print(f"h3.latlng_to_cell ({test_lat},{test_lon},{hexagon_resolution:02d}): {h3_address} ")


h3.latlng_to_cell (43.0375919,-76.135058,09): 892b8a2e3cfffff 


As we have seen before H3 Cell did consist of two parts: first 3 encodes and the remaining encodes the hierarchy.
<br> Let's find out the boundary of 89283470d93ffff

In [34]:
# Boundary of H3 Cell
h3_cell_boundary = h3.cell_to_boundary(h3_address)
for i in range(len(h3_cell_boundary)):
    print(f"v[{i+1}]={h3_cell_boundary[i]}")

v[1]=(43.03906983955765, -76.13379936708955)
v[2]=(43.03806660158603, -76.13596540836582)
v[3]=(43.0362591407131, -76.13580815975423)
v[4]=(43.035454908732, -76.13348504597)
v[5]=(43.036458071661094, -76.13131907499339)
v[6]=(43.03826554161082, -76.13147614749984)


Because this H3 Cell is hexagon, it has 6 vertices. <br> Let's display H3 Cell information on a map.

In [35]:
m = visualize_hexagons([h3_address],color="red")
display(m)

Center :[43.03714745364597, -76.13331032552375]


If the notebook did not show the image, here is the screenshot.
![](m1.png)

Let's also visualize the parent H3 Cell

In [36]:
parents = h3.cell_to_parent(h3_address)
print(f"parent-of({h3_address})={parents}")

parent-of(892b8a2e3cfffff)=882b8a2e3dfffff


In [37]:
childrens =  h3.cell_to_children(parents)
childrens = [h for h in childrens if h!=h3_address]

m = visualize_hexagons(childrens,color="cyan",folium_map=m)

#display(m)

Center :[43.040675316391244, -76.13120273448256]


In [39]:
m = visualize_hexagons([parents],color="blue",folium_map=m)
display(m)

Center :[43.03990126436152, -76.13258401494035]


If the notebook does not show the image, here is the screenshot.
![](./m2.png)

## How to utilize H3 Cells and hierarchical decomposition to implement NNQ?
There are two parameters drive such decision.
 1. Search radius, given in NNQ.
 1. H3 Cell Resolution used when the index structure is built.

For given NNQ and resolution of H3 Cell Index, system needs to figure out which H3 Cells are contained in the circle defined by NNQ.
<br>Then restaurants in the selected H3 Cells are sorted based on L2 distance formula before returning a query result.

In [40]:
neighbors = h3.grid_ring(h3_address)
neighbors

['892b8a2e3c3ffff',
 '892b8a2e3cbffff',
 '892b8a2e353ffff',
 '892b8a2e357ffff',
 '892b8a2e31bffff',
 '892b8a2e3c7ffff']

In [46]:
m = visualize_hexagons(neighbors,color="green")
m = visualize_hexagons([h3_address],color="red",folium_map=m)
m = visualize_hexagons([parents],color="blue",folium_map=m,zoom_start=18)
display(m)

Center :[43.03739617632877, -76.13354654462405]
Center :[43.03714745364597, -76.13331032552375]
Center :[43.03990126436152, -76.13258401494035]


If the notebook does not show the image, here is the screenshot.
![](./m4.png)

The hexagon in red (h3_address) is the H3 Cell that contains query point.
<br> The hexagon in blue is parent H3 Cell of h3_address in the hierarchical decomposition.
<br> The hexagons in green are H3 Cells surrounding h3_address, hence called ring. Based on the given radius and resolution used, it can decide which H3 Cells to leverage for fast NNQ.

## Summary
The objective of this exploration is to demonstrate part of H3 Cell and spatial indexing. Hopefully it helps to understand how it works.